In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
def generate_dummy_images(num_samples=1000, image_size=(64, 64)):
    X = []
    y = []
    for _ in range(num_samples):
        # Malware: noisy/random pixel data
        img = np.random.randint(0, 256, size=image_size).astype('uint8')
        X.append(img)
        y.append(1)

        # Benign: smoother pixel data
        benign_img = np.random.normal(loc=128, scale=30, size=image_size).clip(0, 255).astype('uint8')
        X.append(benign_img)
        y.append(0)
